In [1]:
import numpy as np
import pandas as pd
import PIL

In [3]:
df_images = pd.read_csv('images.csv')
df_products = pd.read_csv('clean_products.csv', lineterminator='\n')

In [4]:
df_all = pd.merge(df_products[['id', 'category_id']], df_images[['product_id', 'id', 'bucket_link']], left_on='id', right_on='product_id').copy()

In [5]:
df_all['img_file_name'] = df_all['bucket_link'].str.split('/').str[-1]
df_all['img_array'] = np.nan

In [6]:
from PIL import Image
from numpy import asarray

df_all['img_array'] = df_all['img_array'].astype(object)

for idx, row in df_all.iterrows():
    img_file = f"./images_clean/{row['img_file_name']}"
    try:
        img = Image.open(img_file)
        
        # convert to array and flatten to 1d
        img_array = asarray(img).flatten()
        img_array = img_array.reshape(img_array.size, 1)
        
        df_all.at[idx, 'img_array'] = img_array

    except FileNotFoundError:
        print(f'No file for {img_file}')

df_all.to_pickle('prod_imgs.pkl')

In [7]:
from sklearn.model_selection import train_test_split
from sklearn import svm, metrics
import pandas as pd


In [ ]:
df_all = pd.read_pickle('prod_imgs.pkl')

In [8]:
# Create a classifier: a support vector classifier
clf = svm.SVC(gamma=0.001)

In [9]:
X = df_all.img_array


y = df_all.category_id

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X_test, X_validate, y_test, y_validate = train_test_split(X_test, y_test, test_size = 0.5)

In [10]:
# Learn the images on the train subset
clf.fit(X_train, y_train)

ValueError: setting an array element with a sequence.

In [ ]:
# Predict the value of the digit on the test subset
predicted = clf.predict(X_test)

In [ ]:
import skimage.io as skio
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score


# Load the data
trainingImages = skio.imread_collection('./images_clean/*.jpg',conserve_memory=True)

# cast to numpy arrays
trainingImages = np.asarray(trainingImages)

# reshape img array to vector
def reshape_image(img):
    return np.reshape(img,len(img)*len(img[0]))

img_reshape = np.zeros((len(trainingImages),len(trainingImages[0])*len(trainingImages[0][0])))

for i in range(0,len(trainingImages)):
    img_reshape[i] = reshape_image(trainingImages[i])

# SVM
clf = svm.SVC(gamma=0.01,C=10,kernel='poly')
clf.fit(img_reshape, trainingTargets.values.tolist())